# Smart-G demo notebook

This is an interactive document allowing to run Smart-G with python and visualize the results. <br>
*Tips*: cells can be executed with shift-enter. Tooltips can be obtained with shift-tab. More information [here](http://ipython.org/notebook.html) or in the help menu. [A table of content can also be added](https://github.com/minrk/ipython_extensions#table-of-contents).

In [ ]:
%pylab inline
# next 2 lines allow to automatically reload modules that have been changed externally
%reload_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
from smartg import Smartg, reptran_merge
from smartg import RoughSurface, LambSurface, FlatSurface, Environment
from smartg import Profile, AeroOPAC, CloudOPAC, IOP_SPM, IOP_MM, REPTRAN, REPTRAN_IBAND, REPTRAN_IBAND_LIST
from tools.tools import SpecInt, SpecInt2, Irr, ReadREPTRAN_bands, reduce_reptran
from tools.luts import LUT, MLUT, Idx, merge, read_lut_hdf, read_mlut
from tools.smartg_view import plot_polar, smartg_view, input_view, mdesc, transect_view
from ipywidgets import interact, interact_manual
from matplotlib import colors
colors_ = list(six.iteritems(colors.cnames))

## How to run Smart-G

In [ ]:
# run SMART-G
# this simulation uses an AFGL tropical atmosphere without aerosols
# Reflectance computation in Cone Sampling mode (default)
m = Smartg().run(THVDEG=30, wl=500., NBPHOTONS=1e8,
           atm=Profile('afglt'),
           surf=RoughSurface(SUR=1))

In [ ]:
# the result is a MLUT
m.print_info()
# it can be written with m.save(filename)
# and then reloaded with read_mlut_hdf(filename)

## How to view the results

In [ ]:
# basic visualization
fig = smartg_view(m)

In [ ]:
# access a LUT within the MLUT
I = m['I_up (TOA)']  # or by index: m[0]
I.desc = mdesc(I.desc) # I descriptor in math Latex format

In [ ]:
I.print_info()

In [ ]:
# access values in this LUT
print I[5, ::5]  # by index (can be float indices to interpolate)
print I[Idx(90.), Idx(45.)]  # or by index finding + interpolation

In [ ]:
# calculate polarized light
# use operations between LUT's, and apply sqrt
Q, U = m['Q_up (TOA)'], m['U_up (TOA)']
LP = (Q*Q + U*U).apply(sqrt, mdesc('LP_up (TOA)'))   # second parameter of apply sets the LUT's desc

In [ ]:
#Different visualizations
LP.plot_semi()
LP.plot()
LP.plot_polar()

In [ ]:
I.plot_semi(index=Idx(45), vmin=0, vmax=0.3)   # the arguments are optional

In [ ]:
# get a sub-LUT from a LUT (here 2D -> 1D)
I.sub()[Idx(90),:].plot()

## A few examples

In [ ]:
# Basic Rayleigh example, no surface, full view
fig= smartg_view(Smartg().run(wl=500., THVDEG=30., NBPHOTONS=1e9, atm=Profile('afglt')), full=True)

In [ ]:
# only Rayleigh with a custom grid, a customes depolarization ratio and a custom surface pressure
fig= smartg_view(Smartg().run(wl=500., THVDEG=30., DEPO=0., NBPHOTONS=1e9,
        atm=Profile('afglt', grid='100[75]25[5]10[0.5]0.5', P0=990.)))

In [ ]:
# Rayleigh + aerosols
aer = AeroOPAC('urban', 0.4, 550.)
pro = Profile('afglms', aer=aer)
fig= smartg_view(Smartg().run(wl=300., THVDEG=60, atm=pro, NBPHOTONS=1e8))

In [ ]:
# atmosphere + surface
fig= smartg_view(Smartg().run(490., NBPHOTONS=1e8, THVDEG=45.
        atm=Profile('afglms'),
        surf=LambSurface(ALB=0.1)))

In [ ]:
# atmosphere + surface + océan
# compute outputs at the surface also, view results for upwelling at 0+
fig= smartg_view(Smartg().run(490., NBPHOTONS=1e8, THVDEG=30., OUTPUT_LAYERS=3,
            atm=Profile('afglms'),
            surf=RoughSurface(),
            water=IOP_MM(1.)), field='up (0+)', logI=True)

In [ ]:
# surface + océan (SPM model)
# compute outputs at the surface also, view results for downwelling at 0-
fig= smartg_view(Smartg().run(380., THVDEG=30., NBPHOTONS=1e8, OUTPUT_LAYERS=3,
        surf=RoughSurface(WIND=15., NH2O=1.34),
        water=IOP_SPM(SPM=0.1)), field='up (0-)')

In [ ]:
# océan seul (chl model)
fig = smartg_view(Smartg().run(wl=380., THVDEG=30., water=IOP_MM(0.1), NBPHOTONS=1e8))

## Multispectral

In [ ]:
# multispectral simulation
pro = Profile('afglt',
              grid=[100, 75, 50, 30, 20, 10, 5, 1, 0.],  # optional, otherwise use default grid
              pfgrid=[100, 20, 0.],   # optional, otherwise use a single band 100-0
              pfwav=[400, 500, 600], # optional, otherwise phase functions are calculated at all bands
              aer=AeroOPAC('maritime_clean', 0.3, 550.),
              verbose=True)
water = IOP_MM(1.,
              pfwav=[400, 500, 600])
m = Smartg().run(wl=np.linspace(400, 600, 11.),
           THVDEG=60., NBPHOTONS=1e8,
           atm=pro,
           surf=RoughSurface(),
           water=water)
m['I_up (TOA)'].sub()[0,:,:].plot_polar()
figure()
m['I_up (TOA)'].sub()[:,Idx(60),Idx(30)].plot(fmt='o-')

## Atmospheric profile

In [ ]:
# monochromatic computation for custom aerosols
wl = 550.
pro = Profile('afglt',
              aer=AeroOPAC('urban',0.3, wl), # set AOT to 0.3 and aerosol vertical profile to 'urban' type
              ssa=0.98,  # set single scattering albedo of aerosols to 0.98 for each layer
              O3 = 0.,   # scale ozone vertical column to 0 Dobson units (here no absoprtion by ozone)
              NO2= False,# disable absorption by NO2
              H2O= 2.,   # scale water vapour column to 2 g/cm-2
              P0 = 980., # set sea level pressure to 980 hPa
              tauR=0.,   # force Rayleigh optical thickness
              grid = [100, 75, 50, 30, 20, 10, 5, 2., 1.15], # set vertical grid, surface altitude at 1.15 km
              )
# import aerosols scattering matrix from external text file with 5 columns:
# angle, (P11+P12)/2, (P11-P12)/2, P33, P43
pro.aer.setphase('/home/did/RTC/SMART-G/validation/opt_kokha_aer_phase.dat')
azimuth_transect = 30. 
_ = smartg_view(Smartg().run(wl=wl, THVDEG=60., atm=pro, NBPHOTONS=1e8), ind=Idx(azimuth_transect), QU=True, logI=True)

## Band gaseous absorption using REPTRAN

### Example 1: Computation of reflectance in MSG-SEVIRI VIS08 channel

In [ ]:
## REPTRAN (example 1)
# REPTRAN k distribution file here MSG/SEVIRI solar channels
SEVIRI_SOLAR = REPTRAN('reptran_solar_msg')

## Select REPTRAN bands  and returning a REPTRAN_IBAND_LIST object (containing all necessary REPTRAN internal bands)
# several ways of selecting bands
# 1) selecting all bands 
ibands = SEVIRI_SOLAR.to_smartg()
# 2) selecting one specific band
ibands = SEVIRI_SOLAR.to_smartg(include='msg1_seviri_ch008')
# 3) selecting all bands that contains "msg1" in the band name
ibands = SEVIRI_SOLAR.to_smartg(include='msg1')
# 4) selecting all bands whose wavelengths of internal bands satisfy the min and max conditions
ibands = SEVIRI_SOLAR.to_smartg(lmax=700.)
# 5) a mix of 3) and 4)
ibands = SEVIRI_SOLAR.to_smartg(include='msg1', lmin=700., lmax=2000.)

surf=RoughSurface(SUR=1, WIND=5., NH2O=1.34) 
atm=Profile('afglms', H2O=4.)

# Run Smart-g for Reptran list of ibands
m1 = Smartg().run(THVDEG=30, wl=ibands.l, NBPHOTONS=1e8, atm=atm, surf=surf, progress=False)

# Postprocessing: regrouping internal bands information into real band (spectral integration)
m1r = reduce_reptran(m1, ibands)

# Plotting bands 
for i,w in enumerate(m1r.axis('Wavelength')):
    # we use for that a subset of m1r MLUT
    _=smartg_view(m1r.sub(d={'Wavelength':i}), ind=Idx(0.))
    print ibands.get_names()[i]

### Example 2: Polarized reflectance in O2A band in sun glint view with Local Estimate

In [ ]:
%%time
# REPTRAN (example 2) Preparation of data
# k distribution file, here full solar channels at coarse resolution
SOLAR_COARSE = REPTRAN('reptran_solar_coarse')
ibands = SOLAR_COARSE.to_smartg(lmin=757., lmax=770.) # within O2A bands

surf=RoughSurface(SUR=1, WIND=5., NH2O=1.34)
atm=Profile('afglms', P0=900., aer=AeroOPAC('continental_average',1.,764.), pfwav=[764.])
# Evaluate reflectance in specific direction Ths=[60.] and RAA=[180.] using LE
# dict containig directions vectors in radians and coded as float32
le={}
le.update(th=np.array([60.],dtype=np.float32)*np.pi/180)
le.update(phi=np.array([180.],dtype=np.float32)*np.pi/180)

Spp=Smartg()

In [ ]:
%%time
# REPTRAN (example 2) Radiative Transfer (first run)
# first run takes longer as it computes all atmospheric absoprtions
m2 = Spp.run(THVDEG=30., wl=ibands.l, NBPHOTONS=1e6, atm=atm, surf=surf, le=le, progress=False)
m2r = reduce_reptran(m2, ibands)

In [ ]:
%%time
# REPTRAN (example 2) Radiative Transfer (other runs)
# it goes faster a slong as atmosphere does not change
for th0 in [0.,30.,60., 75.]:
    m2 = Spp.run(THVDEG=th0, wl=ibands.l, NBPHOTONS=1e6, atm=atm, surf=surf, le=le, progress=False)
    m2r = reduce_reptran(m2, ibands)
    I=m2r['I_up (TOA)']
    Q=m2r['Q_up (TOA)']
    U=m2r['U_up (TOA)']

    LP = (Q*Q + U*U).apply(sqrt, 'LP')
    P = (LP/I*100).apply(abs,'DoLP')
    LP.sub()[:,0,0].plot(fmt='.-',vmin=0, vmax=.15, label=str(th0))
legend(loc='best')

### Example 3: Computation of Sentinel3/OLCI spectrum

In [ ]:
SPP=Smartg()

In [ ]:
# REPTRAN k distribution file here Sentinel 3 solar channels
SENTINEL_SOLAR = REPTRAN('reptran_solar_sentinel')
ibands = SENTINEL_SOLAR.to_smartg(include='olci') # select OLCI bands

surf=RoughSurface(SUR=3, WIND=5., NH2O=1.34) 
atm=Profile('afglms', P0=990., aer=AeroOPAC('desert',0.2, 550.), pfwav=[700.])
water=IOP_SPM(SPM=100.) # very turbid waters
le={}
le.update(th=np.array([30.],dtype=np.float32)*np.pi/180)
le.update(phi=np.array([0, 90., 180.],dtype=np.float32)*np.pi/180)

# Run Smart-g for Reptran list of ibands
m1 = reduce_reptran(SPP.run(THVDEG=60, wl=ibands.l, NBPHOTONS=1e7, le=le, \
                         water=water, atm=atm, surf=surf, progress=True, OUTPUT_LAYERS=3), ibands)

In [ ]:
# Additional runs
# 1 no atmosphere
surf=RoughSurface(SUR=2, WIND=5., NH2O=1.34)
m1_ws = reduce_reptran(SPP.run(THVDEG=60, wl=ibands.l, NBPHOTONS=1e7, le=le, \
                         water=water, atm=None, surf=surf, progress=True, OUTPUT_LAYERS=3), ibands)
surf=RoughSurface(SUR=1, WIND=5., NH2O=1.34) 
# 2 no ocean
m1_as = reduce_reptran(SPP.run(THVDEG=60, wl=ibands.l, NBPHOTONS=1e7, le=le, \
                         water=None, atm=atm, surf=surf, progress=True), ibands)
# just atmosphere
m1_a = reduce_reptran(SPP.run(THVDEG=60, wl=ibands.l, NBPHOTONS=1e7, le=le, \
                         water=None, atm=atm, surf=None, progress=True), ibands)

In [ ]:
# plot the spectra in the principal plane outside glint
m1['I_up (TOA)'].sub()[:,0,0].plot(fmt='.-',vmin=0, vmax=.3, label=r'$TOA \uparrow$')
m1['I_up (0+)'].sub()[:,0,0].plot(fmt='.-',vmin=0, vmax=.3,  label=r'$0^+ \uparrow$')
m1_ws['I_up (0+)'].sub()[:,0,0].plot(fmt='.-',vmin=0, vmax=.3,  label=r'$0^+ \uparrow$, no atm.')
m1_as['I_up (TOA)'].sub()[:,0,0].plot(fmt='.-',vmin=0, vmax=.3, label=r'$TOA \uparrow$, no water')
m1_a['I_up (TOA)'].sub()[:,0,0].plot(fmt='.-',vmin=0, vmax=.3, label=r'$TOA \uparrow$,'+' \n no water, no glint')
legend()

## Fluxes

In [ ]:
# Fluxes in the PAR spectral range for different levels in the ocean atmosphere system
# SUR=3 for letting reflection and transmission at interface
Spp=Smartg()
surf=RoughSurface(SUR=3, WIND=10., NH2O=1.34)
atm=Profile('afglms', H2O=4.)
water=IOP_MM(1, pfwav=[550.]) # Morel and Maritorena water IOPs , phase function computed only at 550. nm
wl = np.linspace(400.,700.,num=11)
# planar and spherical fluxes and OUTPUT_LAYERS=3 for intermediate output levels
P = Spp.run(THVDEG=45, wl=wl, NBPHOTONS=1e6, atm=atm, surf=surf, water=water, 
                 flux='planar', OUTPUT_LAYERS=3)
S = Spp.run(THVDEG=45, wl=wl, NBPHOTONS=1e6, atm=atm, surf=surf, water=water, 
                 flux='spherical', OUTPUT_LAYERS=3)
# plot downward fluxes spectra just underwater after summing in both directions
P['flux_down (0-)'].plot(fmt='.-',vmin=0,vmax=1.,label='down 0- planar')
S['flux_down (0-)'].plot(fmt='.-',vmin=0,vmax=1.,label='down 0- spherical')
legend(loc='best')
# plot upward fluxes spectra just underwater after summing in both directions
figure()
P['flux_up (0-)'].plot(fmt='.-',vmin=0,vmax=0.05,label='up 0- planar')
S['flux_up (0-)'].plot(fmt='.-',vmin=0,vmax=0.05,label='up 0- spherical')
legend(loc='best')

## Looping over parameters

In [ ]:
# loop over some wavelengths
MLUTS = []
Spp=Smartg()
for aot in linspace(0, 1.5, 5):
    M = Spp.run(THVDEG=30,
               wl=443., NBPHOTONS=1e7,
               atm=Profile('afglt',aer=AeroOPAC('desert',aot,443.)),
               surf=RoughSurface(WIND=5.))
    M.set_attr('AOT', aot)
    MLUTS.append(M)

In [ ]:
M = merge(MLUTS, ['AOT'], dtype=float)

In [ ]:
M[0].sub()[:,Idx(90),Idx(45)].plot(fmt='o-')

## Difference between two LUTs

In [ ]:
# Here we compare the TOA radiance simulated with
# plane parallel and spherical atmospheres, at 443 nm.
results = []
for pp in [True, False]:
    results.append(Smartg(pp=pp).run(THVDEG=60, wl=443., NBPHOTONS=1e9,
                          NBTHETA=20., NBPHI=20.,
                          atm=Profile('afglt')))
    
PP = results[0]['I_up (TOA)']
SP = results[1]['I_up (TOA)']

In [ ]:
# show relative difference
plot_polar(100.*(SP-PP)/PP, Idx(175.), fig=figure(figsize(8, 8)), vmin=-20, vmax=20)

## Interactive simulation

In [ ]:
from IPython.html.widgets.interaction import interact, interact_manual

S=Smartg()
le={}
le={}
le.update(phi=np.array([0.]),dtype=np.float32)
le.update(thv=np.array([60.])*np.pi/180,dtype=np.float32)

def simulate(thvdeg, surface, aerosol_model, aot550):
    surf = {True:RoughSurface(), False:None}[surface]
    aer = AeroOPAC(aerosol_model, aot550, 550.)
    print 'TOA Intensity : %.5f'%S.run(THVDEG=thvdeg,
             wl=443., NBPHOTONS=1e5,le=le,
             atm=Profile('afglt', aer=aer), surf=surf, OUTPUT_LAYERS=3, progress=False
          )['I_up (TOA)'].data

interact_manual(simulate, thvdeg=(0, 90), surface=True,
                aerosol_model=AeroOPAC.listStandardAerosolFiles(), aot550=(0.001, 5., 0.01)
                )